In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings("ignore")

diamonds = sns.load_dataset("diamonds")

In [2]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [3]:
diamonds.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [4]:
diamonds.shape

(53940, 10)

In [5]:
diamonds.describe(exclude=['number'])

,cut,color,clarity
count,53940,53940,53940
unique,5,7,8
top,Ideal,G,SI1
freq,21551,11292,13065


In [6]:
from sklearn.model_selection import train_test_split

x,y = diamonds.drop('price', axis=1), diamonds['price']

In [7]:
x

,carat,cut,color,clarity,depth,table,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,6.15,6.12,3.74


In [8]:
y

0         326
1         326
2         327
3         334
4         335
         ... 
53935    2757
53936    2757
53937    2757
53938    2757
53939    2757
Name: price, Length: 53940, dtype: int64

In [9]:
categorical_col = x.select_dtypes(exclude=np.number).columns.tolist()

for col in categorical_col:
    x[col] = x[col].astype('category')

In [10]:
x.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
x           float64
y           float64
z           float64
dtype: object

In [23]:
y.dtypes

dtype('int64')

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

Use the XGBoost data format as DMatrix (can input dataframe into xgb.DMatrix)

In [12]:
import xgboost as xgb

dtrain_reg = xgb.DMatrix(x_train, label=y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(x_test, label=y_test, enable_categorical=True)

In [13]:
params = {
    'objective': 'reg:squarederror',
    'tree-method': "gpu_hist",
}

In [14]:
model = xgb.train(
    params, 
    dtrain=dtrain_reg,
    num_boost_round=100,
)

In [15]:
from sklearn.metrics import root_mean_squared_error

preds = model.predict(dtest_reg)

In [16]:
rmse = root_mean_squared_error(y_test, preds)

print(f"RMSE of the base model: {rmse:.3f}")


RMSE of the base model: 552.861


Using validation sets

In [17]:
evals = [(dtrain_reg, 'train'), (dtest_reg, 'validation')]

In [18]:
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=100,
   evals=evals,
   verbose_eval=10
)

[0]	train-rmse:2874.49146	validation-rmse:2817.90814
[10]	train-rmse:548.36512	validation-rmse:592.03160
[20]	train-rmse:491.09887	validation-rmse:558.53485
[30]	train-rmse:469.58201	validation-rmse:555.51015
[40]	train-rmse:454.32953	validation-rmse:554.45666
[50]	train-rmse:438.68033	validation-rmse:554.13365
[60]	train-rmse:425.38361	validation-rmse:551.57888
[70]	train-rmse:414.71115	validation-rmse:549.26109
[80]	train-rmse:405.41008	validation-rmse:549.03952
[90]	train-rmse:391.04269	validation-rmse:551.87206
[99]	train-rmse:383.48826	validation-rmse:552.86131


In [19]:
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=5000,
   evals=evals,
   verbose_eval=250,
   early_stopping_rounds=20
)

[0]	train-rmse:2874.49146	validation-rmse:2817.90814
[58]	train-rmse:427.93117	validation-rmse:552.90857


Executing cross validation

In [20]:
model = xgb.cv(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=5000,
   nfold = 5,
   early_stopping_rounds=20
)

In [21]:
model.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2874.224552,9.424846,2876.318793,36.995997
1,2088.350837,7.595382,2093.063623,25.351925
2,1552.629638,4.974140,1560.552731,19.550836
3,1185.994963,4.133544,1198.669943,14.648669
4,943.402904,4.757288,962.349383,11.724038


In [22]:
best_rmse = model['test-rmse-mean'].min()

best_rmse

np.float64(550.2735543625861)